# verl rl数据处理

In [ ]:

import os
import datasets
import re
from verl.utils.hdfs_io import copy, makedirs
# To extract the solution for each prompts in the dataset
# def extract_solution(solution_str):
# ...

local_dir = "./process_gsm8k"
num_few_shot = 5
# data_source = 'openai/gsm8k'
data_source = "xxx"

def extract_solution(solution_str):
    solution = re.search("#### (\\-?[0-9\\.\\,]+)", solution_str) # extract the solution after ####
    assert solution is not None
    # #### 72
    final_solution = solution.group(0)
    final_solution = final_solution.split('#### ')[1].replace(',', '')
    return final_solution

instruction_following = "Let's think step by step and output the final answer after \"####\"."

# add a row to each data item that represents a unique id
def make_map_fn(split):

    def process_fn(example, idx):
        # example: {question:xxx answer:xxx}
        # {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}
        question = example.pop('question')
        question = question + ' ' + instruction_following
        # print("answer:", example["answer"])
        answer = example.pop('answer')
        solution = extract_solution(answer)
        # print("solution:",solution)
        data = {
            "data_source": data_source,
            "prompt": [{
                "role": "user",
                "content": question
            }],
            "ability": "math",
            "reward_model": {
                "style": "rule",
                "ground_truth": solution
            },
            "extra_info": {
                'split': split,
                'index': idx
            }
        }
        return data

    return process_fn

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 本地文件===
# test_rata_data_dir = "./process_gsm8k/train.parquet"
# dataset = datasets.load_dataset("parquet", data_files=test_rata_data_dir) 
# print(dataset)
# 加载main子文件夹的===
dataset = datasets.load_dataset(data_source,"main")
"""
train_dataset = Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})
-
gsm8k/main/
    test-00000-of-00001.parquet
    train-00000-of-00001.parquet
"""
train_dataset = dataset['train']
test_dataset = dataset['test']
 
    # Construct a `def make_map_fn(split)` for the corresponding datasets.
# ...

train_dataset = train_dataset.map(function=make_map_fn('train'), with_indices=True)
test_dataset = test_dataset.map(function=make_map_fn('test'), with_indices=True)

train_dataset.to_parquet(os.path.join(local_dir, 'train.parquet'))
test_dataset.to_parquet(os.path.join(local_dir, 'test.parquet'))


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:  32%|███▏      | 2385/7473 [00:00<00:00, 23098.47 examples/s]

answer: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
solutionL: 72
answer: Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10
solutionL: 10
answer: In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5
solutionL: 5
answer: Maila read 12 x 2 = <<12*2=24>>24 pages today.
So she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.
There are 120 - 36 = <<120-36=84>>84 pages left to be read.
Since she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages.
#### 42
solutionL: 42
answer: He writes each friend 3*2=<<3*2=6>>6 pages a week
So he writes 6*2=<<6*2=12>>12 pages every week
That means he writes 12*52=<<12*52=624>>624 pages a year
####

Map: 100%|██████████| 7473/7473 [00:00<00:00, 20672.17 examples/s]


answer: He buys 4 bags of frozen vegetables for $2.00 a bag so that’s 4*2 = $<<4*2=8.00>>8.00
The meat is $5.00, the crackers are $3.50, the vegetables are $8.00 and the cheese is $3.50 for a total of 5+3.50+8+3.50 = $<<5+3.5+8+3.5=20.00>>20.00
He gets 10% discount on his $20.00 purchase so that’s .10*20 = $<<20*.10=2.00>>2.00 off
His final grocery bill will be 20-2 = $<<20-2=18.00>>18.00
#### 18
solutionL: 18
answer: She drew 24 on Saturday and 16 on Sunday for a total of 24+16 = <<24+16=40>>40 drawings
She charges $20.00 per drawing and she drew 40 caricatures so she made $20*40 = $<<20*40=800>>800
#### 800
solutionL: 800
answer: Let x represent the number of Oreos Jordan has
James:3+2x Oreos
Total:3+2x+x=36
3x+3=36
3x=33
x=<<11=11>>11 Oreos
#### 11
solutionL: 11
answer: She eats 3 beignets every morning and there are 7 days in a week so she eats 3*7 = <<3*7=21>>21 beignets in a week
She eats 21 beignets in a week so over 16 weeks she will have eaten 21*16 = <<336=336>>336 beignets
#

Map: 100%|██████████| 1319/1319 [00:00<00:00, 24622.08 examples/s]


answer: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
solutionL: 18
answer: It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3
solutionL: 3
answer: The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000
He increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000
So the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000
So he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000
#### 70000
solutionL: 70000
answer: He sprints 3*3=<<3*3=9>>9 times
So he runs 9*60=<<9*60=540>>540 meters
#### 540
solutionL: 540
answer: If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.
If she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-1

Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 937.90ba/s]


564898

In [ ]:
import pandas as pd
data = pd.read_parquet("./process_gsm8k/train.parquet")
# Index(['data_source', 'prompt', 'ability', 'reward_model', 'extra_info'], dtype='object')
print(data.columns)
# [{'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Let\'s think step by step and output the final answer after "####".', 'role': 'user'}]
print(data["prompt"][0])

[{'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Let\'s think step by step and output the final answer after "####".', 'role': 'user'}]


# llamafactory sft数据处理

In [ ]:
import pandas as pd
train_data_dir = "/xxx/gsm8k/main/train-00000-of-00001.parquet"
raw_data = pd.read_parquet(train_data_dir)
print(raw_data.columns, len(raw_data))
res_data = []
prompt = "Let's think step by step and output the final answer after \"####\"."
for i in range(len(raw_data)):
    res_data.append(
        {
            "instruction":prompt + raw_data["question"].iloc[i],
            "input":"",
            "output": raw_data["answer"].iloc[i]
        }
    )


Index(['question', 'answer'], dtype='object') 7473


In [ ]:
print(res_data[11])
import json
with open('/xxx/llamafactory_input/gsm8k_train_7573.jsonl', 'w', encoding='utf-8') as f:
    for item in res_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

{'instruction': 'Tobias is buying a new pair of shoes that costs $95. He has been saving up his money each month for the past three months. He gets a $5 allowance a month. He also mows lawns and shovels driveways. He charges $15 to mow a lawn and $7 to shovel. After buying the shoes, he has $15 in change. If he mows 4 lawns, how many driveways did he shovel?', 'input': '', 'output': 'He saved up $110 total because 95 + 15 = <<95+15=110>>110\nHe saved $15 from his allowance because 3 x 5 = <<3*5=15>>15\nHe earned $60 mowing lawns because 4 x 15 = <<4*15=60>>60\nHe earned $35 shoveling driveways because 110 - 60 - 15 = <<110-60-15=35>>35\nHe shoveled 5 driveways because 35 / 7 = <<35/7=5>>5\n#### 5'}


# r1蒸馏数据处理

请先运行distill_sft_data_from_rl.py生成数据

In [2]:
import json
jsonl_dir = "./distill_data_from_r1.jsonl"
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # 解析每行的 JSON 数据
                obj = json.loads(line)
                data.append(obj)
            except json.JSONDecodeError:
                print(f"解析 JSON 时出错，行内容为: {line}")
    return data
data = read_jsonl(jsonl_dir)
print(len(data))

37292


In [5]:
right_counter_dict = {}
SAMPLE_NUM = 5
save_data = []
for a_data in data:
    question_dix = a_data["idx"]
    prompt = a_data["prompt"]
    generated_text = a_data["generated_text"]
    if a_data["if_right"]==True:
        right = 1
    else:
        right = 0
    if question_dix not in right_counter_dict:
        right_counter_dict[question_dix] = right
    else:
        right_counter_dict[question_dix] += right

    if right == 1:
        save_data.append(
            {
                "instruction":prompt,
                "input":"",
                "output": generated_text
            }
        )
all_right_counter = 0
for k,v in right_counter_dict.items():
    if v == SAMPLE_NUM:
        all_right_counter += 1
print(f"all right rate:{all_right_counter/len(right_counter_dict)}")


all right rate:0.9044235924932976


In [ ]:
print(len(save_data)/ len(data))
print(save_data[111])
with open('/xxx/gsm8k_distill_from_rlmodel_35497.jsonl', 'w', encoding='utf-8') as f:
    for item in save_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

0.9518663520326075
{'instruction': 'Ann\'s favorite store was having a summer clearance. For $75 she bought 5 pairs of shorts for $7 each and 2 pairs of shoes for $10 each. She also bought 4 tops, all at the same price. How much did each top cost? Let\'s think step by step and output the final answer after "####".', 'input': '', 'output': 'Step 1: Calculate the total cost of the shorts.\nAnn bought 5 pairs of shorts at $7 each.\n\\[ 5 \\text{ pairs} \\times \\$7 \\text{ per pair} = \\$35 \\]\n\nStep 2: Calculate the total cost of the shoes.\nAnn bought 2 pairs of shoes at $10 each.\n\\[ 2 \\text{ pairs} \\times \\$10 \\text{ per pair} = \\$20 \\]\n\nStep 3: Add the cost of the shorts and the shoes to find out how much she spent on these items.\n\\[ \\$35 + \\$20 = \\$55 \\]\n\nStep 4: Subtract the total cost of the shorts and shoes from the total amount Ann spent to find out how much she spent on the tops.\nAnn spent a total of $75.\n\\[ \\$75 - \\$55 = \\$20 \\]\n\nStep 5: Divide the 